In [1]:
import sqlite3
import pandas as pd
from miditime.miditime import MIDITime
import datetime
import julian as jl
from midiutil.MidiFile3 import MIDIFile
import os

In [2]:
'''
Combines the Julian date with the TOD that the event occurred and outputs a datetime
'''
def fix_datetime(jul, hour, minute, second):
    #validate input
    if (int(hour) < 0 or int(hour) > 24):
        print("Hour not between 0 and 24 inclusive")
        return
    if (int(minute) < 0 or int(minute) > 60):
        print("Minute not between 0 and 60 inclusive")
        return
    if (second < 0 or second > 60):
        print("Second not between 0 and 60 inclusive")
        return
    #Get Decimal for time of day.
    dec_time = ((int(hour)*60*60) + (int(minute)*60) + second) / 86400
    jul_str = str(jul)
    flt_jul = float(jul_str) + dec_time
    dt = jl.from_jd(round(flt_jul,2), fmt='jd')
    return dt.replace(microsecond=0)

In [3]:
'''
Used to tune the magnitude of the wildfire to a musical note.
'''
def mag_to_pitch_tuned(magnitude, max_fire, min_fire):
    #Logarithmic scale, reverse order
    scale_pct = mymidi.log_scale_pct(min_fire, max_fire, magnitude, True,direction='log')

    # Pick a range of notes. This allows you to play in a key.
    c_major = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

    #Find the note that matches your data point
    note = mymidi.scale_to_note_classic(scale_pct, c_major)

    #Translate that note to a MIDI pitch
    midi_pitch = mymidi.note_to_midi_pitch(note)

    return midi_pitch

In [4]:
'''
Creates and saves the midi file.
'''
def create_midi(note_list):
    track    = 0
    channel  = 0
    time     = 0              # In beats
    tempo    = mymidi.tempo   # In BPM
    MyMIDI = MIDIFile(1)      # One track Midifile.
    MyMIDI.addTempo(track, time, tempo)

    #Add Notes
    for n in note_list:
        MyMIDI.addNote(track, channel, n[1], n[0], n[3], n[2])

    #Save to MIDI file.
    with open(mymidi.outfile, "wb") as output_file:
        MyMIDI.writeFile(output_file)

I'm going to be doing some data sonification/visualization on wildfire data from 1992-2015 from the wildfire dataset found on Kaggle https://www.kaggle.com/rtatman/188-million-us-wildfires

https://www.nwcg.gov/

Data Sonification Inspiration:
<li> Outside Online episode - Dispatches: The Sound of Science
<li> https://www.revealnews.org/blog/turn-your-data-into-sound-using-our-new-miditime-library/

In [5]:
#State we want to gather data from
fire_state = 'MT'

#First year to start gathering wildfire data from
fire_year_start = 2005
#Last year to gather wildfire data from
fire_year_end = 2015

#Minimum Fire size
min_fire_size = 1

#SQL where conditional string
sql_where_clause = "where STATE = '{0}' and FIRE_YEAR >= {1} and FIRE_YEAR <= {2} and FIRE_SIZE >= {3}".format(fire_state, fire_year_start, fire_year_end, min_fire_size)

#Midi Variables
midi_tempo = 120
midi_filename = "wildfire_sonification.mid"
midi_seconds_per_year = 10
midi_starting_octave = 1
midi_octave_range = 7

### Obtain and Clean Dataset

In [6]:
%%capture
#Get Data out of database file.
cnx = sqlite3.connect("{0}/FPA_FOD_20170508.sqlite".format(os.getcwd()))
df_sonify = pd.read_sql_query("select DISCOVERY_DATE, DISCOVERY_TIME, FIRE_SIZE, LATITUDE, LONGITUDE FROM 'Fires' {0} order by DISCOVERY_DATE asc".format(sql_where_clause), cnx)
df_sonify.dropna()

#Get min and max values of FIRE_SIZE for logarithmic scale and tuning of fire size/notes.
c = cnx.cursor()
c.execute("select MAX(FIRE_SIZE) FROM 'Fires' {}".format(sql_where_clause))
max_fire = c.fetchone()[0]
min_fire = min_fire_size

#Fix and combine the date
for index, row in df_sonify.iterrows():
    if (str(row['DISCOVERY_DATE']).lower() == 'none' or str(row['DISCOVERY_TIME']).lower() == 'none'):
        df_sonify.drop(index, inplace=True)
    else:
        df_sonify.loc[index, 'DISCOVERY_DATE'] = fix_datetime(row["DISCOVERY_DATE"], 
                                                              row["DISCOVERY_TIME"][0:2], row["DISCOVERY_TIME"][2:], 00)
'''
Create MIDITime object.
https://github.com/cirlabs/miditime
(Tempo, Filename, Seconds per year, Starting Octave, Octave Range, Start of Epoch Date)
'''
mymidi = MIDITime(midi_tempo, midi_filename, midi_seconds_per_year, midi_starting_octave, 
                  midi_octave_range, df_sonify.iloc[0]['DISCOVERY_DATE'])
        
#Create new column that contains the number of days since the start of the epoch
for index, row in df_sonify.iterrows():
    df_sonify.loc[index, 'DAYS_SINCE_EPOCH'] = mymidi.days_since_epoch(row['DISCOVERY_DATE'])

#Convert DAYS_SINCE_EPOCH to a BEAT to be used in the MIDI file output.
for index, row in df_sonify.iterrows():
    df_sonify.loc[index, 'BEAT'] = mymidi.beat(row['DAYS_SINCE_EPOCH'])

#Tune the fire size value to fit a musical note. 
for index, row in df_sonify.iterrows():
    df_sonify.loc[index, 'SCALED_FIRE_SIZE'] = mag_to_pitch_tuned(row['FIRE_SIZE'], max_fire, min_fire)

### Generate MIDI file

In [7]:
#Generate Note List
start_time = df_sonify['BEAT'][0]
note_list = []
for index, row in df_sonify.iterrows():
    note_list.append([
        row['BEAT'] - start_time,
        int(row['SCALED_FIRE_SIZE']),
        100,  # velocity
        1  # duration, in beats
    ])

#Create MIDI file
create_midi(note_list)